In [2]:
import numpy as np
import tensorflow as tf

In [4]:
t_max = 80
t_values = tf.range(1, t_max + 1, dtype=tf.float32)

# Compute pbacktime(t) for each time period
pbacktime = 550 * tf.pow((1 - 0.025), (t_values - 1))
pbacktime

<tf.Tensor: shape=(80,), dtype=float32, numpy=
array([550.      , 536.25    , 522.8438  , 509.7727  , 497.02838 ,
       484.6027  , 472.48764 , 460.67545 , 449.15857 , 437.92963 ,
       426.98138 , 416.3069  , 405.8992  , 395.75174 , 385.85794 ,
       376.21152 , 366.80624 , 357.6361  , 348.6952  , 339.9778  ,
       331.47836 , 323.19144 , 315.11166 , 307.2339  , 299.553   ,
       292.0642  , 284.7626  , 277.64355 , 270.70248 , 263.9349  ,
       257.33655 , 250.90315 , 244.63057 , 238.51482 , 232.55196 ,
       226.73816 , 221.06972 , 215.54297 , 210.1544  , 204.90054 ,
       199.77803 , 194.7836  , 189.91402 , 185.16617 , 180.537   ,
       176.02359 , 171.623   , 167.33244 , 163.14912 , 159.0704  ,
       155.09364 , 151.21631 , 147.4359  , 143.75002 , 140.15625 ,
       136.65236 , 133.23605 , 129.90517 , 126.65753 , 123.4911  ,
       120.403824, 117.39373 , 114.458885, 111.59742 , 108.80749 ,
       106.087296, 103.43512 , 100.84924 ,  98.32802 ,  95.86982 ,
        93.4730